# Scrape Inspection Details
This notebook obtains the inspection details for each entity

In [11]:
import pandas as pd
import requests
from tqdm import tqdm
import pickle

df = pd.read_pickle('DPH_entities_df.pkl')
df

,dph_id,map_add,name,score,date,place_id,types
0,MTIyOTQyMTA=,"175 MEAD RD, DECATUR, GA 30030",OAKHURST ELEMENTARY SCHOOL,100,03-28-2023,ChIJRfrKdi4H9YgRwOiI0-YFgbI,"[establishment, point_of_interest, primary_sch..."
1,MTIyOTQyOTM=,"3100 MOUNT OLIVE DR, DECATUR, GA 30033",DRUID HILLS MIDDLE SCHOOL,100,03-28-2023,ChIJI-EIKIsH9YgRtfevY250O74,"[establishment, point_of_interest, school]"
2,MTIyOTk3MzY=,"5330 SNAPFINGER WOODS DR, DECATUR, GA 30035","SPORTZ CENTER ACADEMY, INC FAK",90,03-28-2023,ChIJc5eRemSs9YgRlfB8wal4mlQ,"[establishment, point_of_interest]"
3,MTIyOTk4MjU=,"1707 CHURCH STREET, SUITE C-7, DECATUR, GA 30033",CHAT PATTI INDIAN VEGETARIAN RESTAURANT,91,03-28-2023,ChIJ8bIRYDkG9YgR7BY43mZj93Y,"[establishment, food, point_of_interest, resta..."
4,MTIzMDEzMzc=,"4306 LAWRENCEVILLE HIGHWAY, SUITE 140, TUCKER,...",FIVE GUYS,97,03-28-2023,ChIJB4iT1T2m9YgRnW5VlvLqF_I,"[establishment, food, meal_takeaway, point_of_..."
...,...,...,...,...,...,...,...
7867,MzEzNjg1MQ==,"299 NORTH HIGHLAND AVE STE F, ATLANTA, GA 30307",Bread & Butterfly,99,05-28-2020,ChIJHwhNYAEE9YgRCs-R7L-_uWI,"[cafe, establishment, food, point_of_interest,..."
7868,OTQ5MjM3NQ==,"730 BARNETT ST NE STE A, ATLANTA, GA 30306",Plant Based Pizzeria- Mobile,100,05-12-2020,ChIJW31yfH8F9YgRdtbpWzsNBe0,"[establishment, food, point_of_interest, resta..."
7869,Nzk1NjQzMw==,"296 NORTHSIDE DR SE STE B, ATLANTA, GA 30315",Boston Fish Supreme,92,05-11-2020,ChIJ59NmO3AD9YgRJsL3BTxHUVk,"[establishment, food, point_of_interest, resta..."
7870,Nzk1Mzc1MQ==,"1005 CRESTLINE PKWY, ATLANTA, GA 30328",SpringHill Suites By Marriott Atlanta Perimete...,99,05-08-2020,ChIJM9VxgKwO9YgRyAdoKEnhMeU,"[establishment, lodging, point_of_interest]"


In [6]:
extracted = []
for ids in tqdm(df.dph_id.unique()):
    try:
        req_url = f'https://ga.healthinspections.us/stateofgeorgia/API/index.cfm/inspectionsData/{id}'
        r = requests.get(req_url)
        data = r.json()
    except Exception as e:
        print(e, ids)
        data = None
    d = {
        'id': ids,
        'data': data
    }
    extracted.append(d)

# Save our hard-earned data (over an hour of scraping) to pickle
with open('DPH_inspections_raw.pkl', 'wb') as pick:
    pickle.dump(extracted, pick)


100%|██████████| 7872/7872 [32:59<00:00,  3.98it/s]  


# DB Schema: inspections
The following table saves inspection details on a per-inspection basis. (Each row is an inspection)
- inspection_id: \<int> primary key nullable=false
- dph_entity: 'facilityID' \<str> 64bit encoded nullable=false
- date: \<datetime> nullable=false
- purpose: \<str/categorical> nullable=false
- score: \<int> nullable=false
- inspector: \<str> nullable=false
- printable_path: \<str> nullable=false
- volation_count: \<int> nullable=false
- volations: \<list> nullable=true

In [45]:
from datetime import datetime

rows = []
for entity in extracted:
    entity_id = entity['id']
    entity_data = entity['data']
    for inspection in entity_data:
        inspection_data = inspection['columns']
        inspection_id = inspection['inspectionId']
        if inspection_id != 0:
            violation_count = len(inspection['violations'])
            violation_codes = []
            inspector_notes = []
            if violation_count > 0:
                for k, v in inspection['violations'].items():
                    violation_codes.append(v[0].split(' - ', 1)[0])
                    inspector_notes.append(v[5].replace("Inspector Notes: ", ''))
            try:
                row = {
                    'inspection_id': inspection_id,
                    'dph_id': entity_id,
                    'date': datetime.strptime(inspection_data['0'].replace("Date: ", ''), '%m-%d-%Y').date(),
                    'purpose': inspection_data['1'].replace("Inspection Purpose: ", ''),
                    'score': int(inspection_data['2'].replace("Score: ", '')),
                    'inspector': inspection_data['3'].replace("Inspector: ", ''),
                    'printable_path': inspection['printablePath'],
                    'violation_count': violation_count,
                    'violation_codes': violation_codes,
                    'inspector_notes': inspector_notes
                }
            except Exception as e:
                print(e, inspection, entity)
                break
            rows.append(row)
        else:
            print(entity)
            # These entities don't have inspections on file (perhaps they're new?)

df_inspections = pd.DataFrame.from_records(rows)
# Merge with basic data from part 1.
entity_ids = df[['dph_id', 'name']]
m_df = df_inspections.merge(entity_ids, how='inner', on='dph_id')
m_df.to_pickle('inspection_df.pkl')

{'id': 'MTI3NjYyNzE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3NjYyNzE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI3NzM0NzU=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3NzM0NzU=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTIyOTk3MzE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTIyOTk3MzE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI3Njg4Nzk=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3Njg4Nzk=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI5ODUzMDE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI5ODUzMDE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI5ODUzMDQ=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI5ODUzMDQ=', 'columns': {'0': 'No Inspections To Display'}, 

In [18]:
m_df

,inspection_id,dph_id,date,purpose,score,inspector,printable_path,violation_count,violation_codes,inspector_notes
0,57344281,MTIyOTQyMTA=,2023-03-28,Routine,100,Alaina Mauldin,../../_templates/87/food_2015/_report_full.cfm...,0,[],[]
1,57323814,MTIyOTQyMTA=,2022-11-04,Routine,100,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,0,[],[]
2,57298579,MTIyOTQyMTA=,2022-04-21,Routine,100,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,0,[],[]
3,57269737,MTIyOTQyMTA=,2021-08-23,Routine,89,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,1,[6-1A],[Observed TCS food item (diced tomatoes) not c...
4,57344379,MTIyOTQyOTM=,2023-03-28,Routine,100,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,0,[],[]
...,...,...,...,...,...,...,...,...,...,...
18959,56760259,MzEzNjg1MQ==,2020-05-28,Routine,99,Bresa Daise,../../_templates/87/food_2015/_report_full.cfm...,1,[17D],[Observed JBL speaker on prep top table. All e...
18960,56759669,OTQ5MjM3NQ==,2020-05-12,Routine,100,Bresa Daise,../../_templates/87/food_2015/_report_full.cfm...,0,[],[]
18961,56759654,Nzk1NjQzMw==,2020-05-11,Routine,92,shirleyas chambers,../../_templates/87/food_2015/_report_full.cfm...,3,"[2-2D, 11C, 17D]","[At the time of inspection, observed no handso..."
18962,56759650,Nzk1Mzc1MQ==,2020-05-08,Routine,99,Richard Campbell,../../_templates/87/food_2015/_report_full.cfm...,2,"[17C, 17D]","[Ceiling not smooth and easily cleanable., Obs..."


# DB Schema: Violations
The following table saves inspection details on a per-violation basis (each row is a specific violation). In theory,
this will allow every kind of query/aggregation.
- violation_id: \<index>
- inpsection_id: \<int>
- facility_id: \<str>
- violation_desc: \<str>
- points: \<int>
- corrected: \<bool>
- repeat: \<bool>
- notes: \<str>

In [68]:
from datetime import datetime

rows = []
for entity in extracted:
    entity_id = entity['id']
    entity_data = entity['data']
    for inspection in entity_data:
        inspection_data = inspection['columns']
        inspection_id = inspection['inspectionId']
        if inspection_id != 0:
            violation_count = len(inspection['violations'])
            if violation_count > 0:
                for k, v in inspection['violations'].items():
                    violation_raw = v[0].split(' - ', 1)
                    violation_code = violation_raw[0]
                    section_detail = violation_raw[1]
                    inspector_note = v[5].replace("Inspector Notes: ", '')
                    corrected = v[3].replace("Corrected during inspection?: ", '')
                    repeat = v[4].replace("Repeat: ", '')
                    if "Yes" in corrected:
                        corrected = True
                    else:
                        corrected = False
                    if "Yes" in repeat:
                        repeat = True
                    else:
                        repeat = False
                    try:
                        row = {
                            'inspection_id': inspection_id,
                            'dph_id': entity_id,
                            'date': datetime.strptime(inspection_data['0'].replace("Date: ", ''), '%m-%d-%Y').date(),
                            'purpose': inspection_data['1'].replace("Inspection Purpose: ", ''),
                            'score': int(inspection_data['2'].replace("Score: ", '')),
                            'inspector': inspection_data['3'].replace("Inspector: ", ''),
                            'printable_path': inspection['printablePath'],
                            'violation_count': violation_count,
                            'violation_code': violation_code,
                            'section_detail': section_detail,
                            'inspector_note': inspector_note,
                            'corrected': corrected,
                            'repeat': repeat
                        }
                    except Exception as e:
                        print(e, inspection, entity)
                        break
                    rows.append(row)
        else:
            print(entity)

df_violations = pd.DataFrame.from_records(rows)
entity_details = df[['dph_id', 'name', 'types', 'place_id']]
merged_df = df_violations.merge(entity_details, how="inner", on="dph_id")
merged_df.to_pickle('violations_df.pkl')

{'id': 'MTI3NjYyNzE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3NjYyNzE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI3NzM0NzU=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3NzM0NzU=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTIyOTk3MzE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTIyOTk3MzE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI3Njg4Nzk=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI3Njg4Nzk=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI5ODUzMDE=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI5ODUzMDE=', 'columns': {'0': 'No Inspections To Display'}, 'violations': {}}]}
{'id': 'MTI5ODUzMDQ=', 'data': [{'printablePath': '', 'inspectionId': 0, 'facilityId': 'MTI5ODUzMDQ=', 'columns': {'0': 'No Inspections To Display'}, 

In [16]:
merged_df

,inspection_id,dph_id,date,purpose,score,inspector,printable_path,violation_count,violation_code,section_detail,inspector_note,corrected,repeat,name,types,place_id
0,57269737,MTIyOTQyMTA=,2021-08-23,Routine,89,Mariam Musa,../../_templates/87/food_2015/_report_full.cfm...,1,6-1A,proper cold holding temperatures,Observed TCS food item (diced tomatoes) not co...,True,True,OAKHURST ELEMENTARY SCHOOL,"[establishment, point_of_interest, primary_sch...",ChIJRfrKdi4H9YgRwOiI0-YFgbI
1,57317194,MTIyOTQyOTM=,2022-09-21,Routine,95,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,3,4-2B,food-contact surfaces: cleaned & sanitized,Observed two sanitizer buckets reading below 2...,True,False,DRUID HILLS MIDDLE SCHOOL,"[establishment, point_of_interest, school]",ChIJI-EIKIsH9YgRtfevY250O74
2,57317194,MTIyOTQyOTM=,2022-09-21,Routine,95,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,3,15A,"food and nonfood-contact surfaces cleanable, p...",Observed rug on underneath rubber mat on floor...,True,False,DRUID HILLS MIDDLE SCHOOL,"[establishment, point_of_interest, school]",ChIJI-EIKIsH9YgRtfevY250O74
3,57317194,MTIyOTQyOTM=,2022-09-21,Routine,95,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,3,15C,nonfood-contact surfaces clean,Observed icicles that had formed on walk in fr...,False,False,DRUID HILLS MIDDLE SCHOOL,"[establishment, point_of_interest, school]",ChIJI-EIKIsH9YgRtfevY250O74
4,57293839,MTIyOTQyOTM=,2022-03-15,Routine,91,Carley Rhoden,../../_templates/87/food_2015/_report_full.cfm...,1,6-1B,proper hot holding temperatures,Observed multiple foods holding below 135F. Se...,True,False,DRUID HILLS MIDDLE SCHOOL,"[establishment, point_of_interest, school]",ChIJI-EIKIsH9YgRtfevY250O74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38249,56759654,Nzk1NjQzMw==,2020-05-11,Routine,92,shirleyas chambers,../../_templates/87/food_2015/_report_full.cfm...,3,2-2D,adequate handwashing facilities supplied & acc...,"At the time of inspection, observed no handsoa...",True,False,Boston Fish Supreme,"[establishment, food, point_of_interest, resta...",ChIJ59NmO3AD9YgRJsL3BTxHUVk
38250,56759654,Nzk1NjQzMw==,2020-05-11,Routine,92,shirleyas chambers,../../_templates/87/food_2015/_report_full.cfm...,3,11C,approved thawing methods used,Observed chicken in meat sink thawing in stagn...,True,False,Boston Fish Supreme,"[establishment, food, point_of_interest, resta...",ChIJ59NmO3AD9YgRJsL3BTxHUVk
38251,56759654,Nzk1NjQzMw==,2020-05-11,Routine,92,shirleyas chambers,../../_templates/87/food_2015/_report_full.cfm...,3,17D,adequate ventilation and lighting; designated ...,At the time of inspection observed employee dr...,True,False,Boston Fish Supreme,"[establishment, food, point_of_interest, resta...",ChIJ59NmO3AD9YgRJsL3BTxHUVk
38252,56759650,Nzk1Mzc1MQ==,2020-05-08,Routine,99,Richard Campbell,../../_templates/87/food_2015/_report_full.cfm...,2,17C,"physical facilities installed, maintained, and...",Ceiling not smooth and easily cleanable.,False,False,SpringHill Suites By Marriott Atlanta Perimete...,"[establishment, lodging, point_of_interest]",ChIJM9VxgKwO9YgRyAdoKEnhMeU


In [20]:
# The most common violations:
merged_df.groupby('violation_code')['violation_count'].sum()

violation_code
1-2A     2862
1-2B     3719
10A         4
10B        32
10C        37
10D      6614
11A      5811
11B        17
11C      2593
11D      2291
12A      5715
12B      5771
12C      3028
12D       772
13A      9022
13B       135
14A      4909
14B      2992
14C      2779
14D        98
15A      8704
15B      4464
15C     12959
16A      1825
16B      3299
16C       381
17A      1912
17B      1629
17C     13128
17D      6662
18       9582
2-1B     2093
2-1C      509
2-2A     4662
2-2B     2819
2-2C        3
2-2D    13440
2-2E     4518
3-1A      111
3-1B       76
3-1C     1375
3-1D      102
4-1A     4415
4-1B      104
4-2A     5499
4-2B    20810
5-1A      126
5-1B      218
5-2       644
6-1A    10140
6-1B     3306
6-1C     1234
6-1D     1826
6-2      5584
7-1        23
8-2B     4645
9-2       142
Name: violation_count, dtype: int64